# VGCC Simulation

In [15]:
import os
import csv
import numpy as np
import pandas as pd
from markov_model import read_json_model, MarkovModel
from channels import get_output_values, generate_piecewise_linear_string, evaluate_piecewise_string, extract_toggle_times
import plotly.graph_objects as go
import plotly.subplots as sp

Model Specifications

In [16]:
# Specifications
channel_names = ['PQ', 'N', 'R']
n_simulations = 1000 # Number of Monte Carlo Simulations to run

# Prepare stimulus
stim_df = pd.read_csv(os.path.join('data','voltage','AP_Single.csv'), header=None)
stim_df.columns = ['Time (ms)', 'Voltage (mV)']
stimuli = {
    'V' : {
        'timestamp' : stim_df['Time (ms)'].to_numpy(),
        'value' : stim_df['Voltage (mV)'].to_numpy(),
        'time_units' : 'ms',
        'stim_units' : 'mV'
    }
}

Run Stochastic Simulations And Export Channel Opening Dynamics

In [17]:
for name in channel_names:

    # Define channel model from file specifications
    model = MarkovModel(**read_json_model(os.path.join('models', 'vgccs', f'{name}.json')))

    # Run simulation saving individual sample paths
    print(f"Simulating {name} channel...")
    simulation = model.simulate(stimuli=stimuli, mode='stochastic', n_simulations=n_simulations, save_paths=True)
    print(f"Done. Exporting data...")

    # Extract open / closed toggle times
    toggle_times = extract_toggle_times(model, simulation) # in milliseconds

    # Export to csv
    export_path = os.path.join('data', 'calcium', f'{name}_stochastic_toggle_times.csv')
    if not os.path.exists(os.path.dirname(export_path)):
        os.makedirs(export_path)
    with open(export_path, 'w', newline="") as f:
        csvwriter = csv.writer(f)
        for row in toggle_times:
            row = [time / 1000 for time in row] # convert to seconds for export
            csvwriter.writerow(row)

    print('Done.')

Simulating PQ channel...
Done. Exporting data...
Done.
Simulating N channel...
Done. Exporting data...
Done.
Simulating R channel...
Done. Exporting data...
Done.


Run Deterministic simulations

In [18]:
deterministic_simulations = []
for name in channel_names:

    # Define channel model from file specifications
    model = MarkovModel(**read_json_model(os.path.join('models', 'vgccs', f'{name}.json')))

    # Run simulation
    print(f"Simulating {name} channel...")
    deterministic_simulations.append(model.simulate(stimuli=stimuli, mode='deterministic'))
    print(f"Done.")

Simulating PQ channel...
Done.
Simulating N channel...
Done.
Simulating R channel...
Done.


Convert channel probabilities to piecewise linear currents

In [19]:
# Define conductance model

g = 2.7 # pS
v_reversal = 55 #mV

conductance = {
    'timestamp': stimuli['V']['timestamp'] / 1000, # (s)
    'value': - g * (stimuli['V']['value'] - v_reversal) / 1000 # (pA). Negative to make currents positive.
}

In [20]:
# Convert to piecewise linear strings and export to csv

for name in channel_names:
    
    # Load channel open toggle data
    toggles = []
    with open(os.path.join('data', 'calcium', f'{name}_stochastic_toggle_times.csv'), 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            # Convert the strings to floats
            float_row = [float(x) for x in row]
            toggles.append(float_row)
    
    # Convert to strings and export
    export_path = os.path.join('data', 'calcium', f'{name}_stochastic_current_strings.csv')
    if not os.path.exists(os.path.dirname(export_path)):
        os.makedirs(export_path)
    with open(export_path, 'w', newline="") as f:
        csvwriter = csv.writer(f)
        for row in toggles:
            func_string = generate_piecewise_linear_string(row, conductance, 0)
            csvwriter.writerow([func_string])

Extract and Visualise results

In [21]:
# Calculate average open probability from exported files
sample_times = np.linspace(0, stimuli['V']['timestamp'][-1]/1000, 1000)
channel_probs = []
for name in channel_names:
    
    mean_prob = np.zeros(len(sample_times))
    data = []
    with open(os.path.join('data', 'calcium', f'{name}_stochastic_toggle_times.csv'), 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            # Convert the strings to floats
            float_row = [float(x) for x in row]
            data.append(float_row)

        for count, sample_toggles in enumerate(data):
            mean_prob += get_output_values(sample_toggles[1:], sample_times, 0)
        mean_prob /= (count+1)
    
    channel_probs.append(mean_prob)

# Convert to current
channel_currents = []
for mean_prob in channel_probs:
    channel_currents.append(np.interp(sample_times, conductance['timestamp'], conductance['value']) * mean_prob) # pA

# Plot results
colours = ['red', 'green', 'blue']
fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)
fig.add_trace(
    go.Scatter(x=stimuli['V']['timestamp']/1000, y=stimuli['V']['value'], mode='lines', line=dict(color='black'), name='Voltage Stimulus'),
               row=1, col=1
)
fig.update_yaxes(title_text='Voltage (mV)', row=1, col=1)

for mean_prob, current, name, colour in zip(channel_probs, channel_currents, channel_names, colours):
    fig.add_trace(
        go.Scatter(x=sample_times, y=mean_prob, mode='lines', line=dict(color=colour), name=name),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=sample_times, y=current, mode='lines', line=dict(color=colour), name=name, showlegend=False),
        row=3, col=1
    )

fig.update_xaxes(title_text='Time (s)', ticks='inside', linecolor='black', row=3, col=1)
fig.update_yaxes(title_text='Open Probability', ticks='inside', linecolor='black', row=2, col=1)
fig.update_yaxes(title_text='Current Magnitude (pA)', row=3, col=1)

for row in range(1,4):
    fig.update_layout(hovermode='x', plot_bgcolor='white') # template = 'plotly_dark'
    fig.update_yaxes(ticks='inside', linecolor='black', row=row, col=1)

fig.update_layout(height=900, width=900, title='VGCC Activation')
fig.show()